# What to do now

Go through this notebook. Check that it runs through, remove redundant cells, mold it so that it is easy to read.

Tests that should be implemented here

... for SnapshotMatcher:
- matcher output is of correct shape and type
- ... and the SUBFIND subgroups of linked subhalos seem reasonable on first inspection
- ... and there are enough matches
- matcher runtime is acceptable
- matches_ref and matches_srch are compatible
- when matching one-to-one, no subhalos are matched with more than one other subhalo

... for MergerTree:
- build_tree output is of correct shape and type
- runtime is acceptable
- when matching one-to-one, no subhalos are matched with more than one other subhalo between two snapshots
- masses of unlinked subhalos
- (trajectories of randomly selected subhalos seem reasonable)

---

After this notebook is polished, check the following notebooks:
- central evolution plots
- satellite trajectories and mass evolution

These are the smoking-gun tests for the merger tree. In the above notebooks, try to use low level methods (e.g. avoid the Subhalo object for now) s.t. the tests are directed to the essential objectives.

If all these tests check out, the simtrace and matcher modules can be trusted quite safely. Then, I can make a git commit for the changes to these modules and these test notebooks.

---

What then follows, is testing for the Subhalo class, subhalo tracing, fall-in times, data values at fall-in, etc. This should be carefully tested in specific notebooks.

## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
from astropy.cosmology import FlatLambdaCDM, z_at_value

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import simulation
import simtrace_redo
import match_halo_redo
import dataset_comp
import subhalo

In [ ]:
import importlib
importlib.reload(simulation)
importlib.reload(simtrace_redo)
importlib.reload(match_halo_redo)
importlib.reload(dataset_comp)
importlib.reload(subhalo)

For computation time efficiency analysis:

In [ ]:
%load_ext line_profiler

# Preliminary Tests for Simulation Tracing

## Test match_halo

Set n_link_ref to the same value as the SUBFIND subhalo minimum particle limit. With n_matches=1, the linking is done injectively.

In [ ]:
matcher = match_halo_redo.SnapshotMatcher(n_link_ref=20, n_matches=1)

First, study the low-resolution simulation. Create new, blank envelope files in the directory "test_tracing" (make sure that no envelope files exist in that directory before testing). 

In [ ]:
sim_id = "V1_LR_fix"
env_path = os.path.abspath(os.path.join('..', 'test_tracing_inj'))
print(env_path)
sim = simulation.Simulation(sim_id, env_path=env_path)
sim.get_snap_ids()

Try matching two snapshots. Inspect the runtime.

In [ ]:
# Branching backward in time:
snap_ref = sim.get_snapshot(126)
snap_srch = sim.get_snapshot(127)
matches_ref, matches_srch = matcher.match_snapshots(snap_ref, snap_srch)
print(matches_ref.shape)
print(matches_srch.shape)

In [ ]:
%lprun -f matcher.match_injective matcher.match_snapshots(snap_ref, snap_srch)

With ~3s matching time for a pair of snapshots and up to ~120 snapshots to match, the total runtime for tracing the entire LR simulation would be around 6 min. This seems acceptable.

Of course, the question is: is this fast enough to trace a MR simulation? The most computing time is spent in the `np.interset1d` function calls, i.e. computing the intersections between the most bound particles between two subhalos. This function is called 113907 times, which means that 113907 pairs of subhalos are tried as matches. 

This is probably the most time efficient function for finding the intersections, i.e. finding out whether two subhalos match. Thus, the only way to speed up the `match_snapshots` function would be to decrease the number of matching trials. But this is also quite difficult to do: probably the most promising way would be to use spatial and kinematic information. 

Note that the number of matching trials, if every subhalo in snap_ref was tried with every subhalo in snap_srch, would be 1116 * 1125 = 1255500. So, by restring trials to subhalos within mass range of factor 3 and to unmatched subhalos in srch, and by halting at the first match, we reduce the number of trials by about 90 % (1 - 113907 / (1116 * 1125) = 0.909). Most subhalos are in the low-mass range. Also, the subhalos with the smallest mass are also less likely to be matched at all.

One simple way to speed up this function appears to be to match subhalos in the original order, by SUBFIND subgroup. However, then the matches are not necessiraly made between the most massive subhalos (of multiple potential match candidates). While it is rare that more than one such candidate would even exist, the time advantage is only of some factor between 1-10, so I stick with the mass ordering.

Let
> $n$ be the average number of subhalos in a snapshot, and \
> $k$ the average number of bound particles in a subhalo.

The time complexity of `match_snapshots` is $\mathcal{O}(n^2)$. 

### Inspect the matches

First, simply print the matches:

In [ ]:
gns_ref = snap_ref.get_subhalos('GroupNumber')
sgns_ref = snap_ref.get_subhalos('SubGroupNumber')
gns_srch = snap_srch.get_subhalos('GroupNumber')
sgns_srch = snap_srch.get_subhalos('SubGroupNumber')

for i, j in enumerate(matches_ref):
    if j != matcher.no_match:
        print("({}, {}) --> ({}, {})".format(gns_ref[i], sgns_ref[i],
                                             gns_srch[j], sgns_srch[j]))
    else:
        print("({}, {}) NO MATCH".format(gns_ref[i], sgns_ref[i]))

Most massive central halos are always matched. Most of the subhalos of M31 and MW are matched. 

Note, that subhalos of a central remain subhalos of the same central, even when the group number changes:
> (10.0, 0.0) --> (11.0, 0.0) \
> (10.0, 1.0) --> (11.0, 1.0) \
> (10.0, 2.0) --> (11.0, 2.0) \
> (11.0, 0.0) --> (9.0, 0.0) \
> (11.0, 1.0) --> (9.0, 1.0) \
> (11.0, 2.0) --> (9.0, 3.0)

Or, even:
> (18.0, 0.0) --> (2.0, 3.0) \
> (18.0, 1.0) --> (2.0, 26.0) \
> (18.0, 2.0) --> (2.0, 46.0)

This is assuring: at least, it appears I manage to link the more massive subhalos of the simulation. But also, most of the least massive ones are linked, which is to be expected, since the SUBFIND 20 minimum particle limit is designed to rule out most spurious numerical structures. It is still left unclear, how long these objects actually survive, however. 

Another thing to note, is that quite many satellites of the M31 and MW analogues are flying out:
> (1.0, 18.0) --> (188.0, 0.0) \
> (2.0, 31.0) --> (415.0, 0.0) \
> (2.0, 38.0) --> (472.0, 0.0)

Invert matches_ref and compare with matches_srch. The output, matches_inv, and matches_srch should be exactly equal.

In [ ]:
def invert_matches(matches, inv_size, no_match):

    # Initialize match array for subhalos in search:
    matches_inv = no_match * np.ones(inv_size, dtype=int)

    # Iterate through matches:
    for sub_idx_ref, sub_idx_srch in enumerate(matches_ref):
        if sub_idx_srch == no_match:
            continue

        if matches_inv[sub_idx_srch] != no_match:
            print("Odd. Appears non-injective...")
        
        matches_inv[sub_idx_srch] = sub_idx_ref
        
    return matches_inv

In [ ]:
matches_inv = invert_matches(matches_ref, matches_srch.size, matcher.no_match)
print(np.sum(matches_inv != matches_srch))

No two subhalos in snap_ref are matched with the same subhalo in snap_srch:

In [ ]:
vals, cnts = np.unique(matches_ref, return_counts=True)
print("Total number of matches: {}".format(vals.size))
print("Subhalos in snap_srch that are matched more than ones:")
print("Indices: {}".format(vals[cnts > 1]))
print("Counts: {}".format(cnts[cnts > 1]))

In [ ]:
vals, cnts = np.unique(matches_srch, return_counts=True)
print("Total number of matches: {}".format(vals.size))
print("Subhalos in snap_srch that are matched more than ones:")
print("Indices: {}".format(vals[cnts > 1]))
print("Counts: {}".format(cnts[cnts > 1]))

... or:

In [ ]:
print(np.sum(matches_srch[:] != matcher.no_match))

## Merger Trees

In [ ]:
snap_start = 101
snap_stop = 128

In [ ]:
sim.get_snapshot(snap_start).group_data.fname

In [ ]:
mtree = simtrace_redo.MergerTree(sim, branching="BackwardBranching")
%lprun -f mtree.build_tree_with_back_branch mtree.build_tree(snap_start, snap_stop, overwrite=True)

Again, no mergers found between any of the pairs of snapshots:

In [ ]:
snap_stop=101
for sid in range(snap_stop, 127):
    snap = sim.get_snapshot(sid)
    desc = snap.get_subhalos('Descendants', mtree.h5_group)
    vals, cnts = np.unique(desc, return_counts=True)
    mask_merger = np.logical_and(vals != mtree.no_match, cnts > 1)
    print(sid, np.sum(mask_merger))
    print(np.size(desc), np.sum(desc != mtree.no_match))

## Unidentified subhalos

At each snapshot, about 300 subhalos are present that have neither identified progenitors or descendants:

In [ ]:
snap_start=102; snap_stop=127
for sid in range(snap_start, snap_stop):
    snap = sim.get_snapshot(sid)
    desc = snap.get_subhalos('Descendants', mtree.h5_group)
    prog = snap.get_subhalos('Progenitors', mtree.h5_group)
    print("Snapshot {}".format(sid))
    print("  # subhalos: {}".format(np.size(desc)))
    print("  # with no progenitor: {}".format(np.sum(prog == mtree.no_match)))
    print("  # with no descendant: {}".format(np.sum(desc == mtree.no_match)))
    mask_shadow = np.logical_and(prog == mtree.no_match, desc == mtree.no_match)
    print("  # with neither: {}".format(np.sum(mask_shadow)))
    
    gns = snap.get_subhalos('GroupNumber')
    sgns = snap.get_subhalos('SubGroupNumber')
    print("    # with SGN=0: {}".format(
        np.sum(np.logical_and(mask_shadow, sgns==0))))
    n = 100
    print("    # with GN>{}: {}".format(n,
        np.sum(np.logical_and(mask_shadow, gns>n))))
    print("")

Let us inspect the masses of these subhalos:

In [ ]:
sid = 126
snap = sim.get_snapshot(sid)
desc = snap.get_subhalos('Descendants', mtree.h5_group)
prog = snap.get_subhalos('Progenitors', mtree.h5_group)
mask_shadow = np.logical_and(prog == mtree.no_match, desc == mtree.no_match)

masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
    snap, (1,0), (2,0))
mask_sat = np.logical_or.reduce(masks_sat)

mass = snap.get_subhalos('Mass') * units.g.to(units.Msun)
m = np.sort(mass)
m_s = np.sort(mass[mask_shadow])

fig, ax = plt.subplots(ncols=2)
ax[0].set_xscale('log')
ax[1].set_xscale('log')

m = mass[mask_sat]
ax[0].plot(np.sort(m), np.arange(m.size))
m = mass[np.logical_and(mask_sat, mask_shadow)]
ax[0].plot(np.sort(m), np.arange(m.size))

m = mass[mask_isol]
ax[1].plot(np.sort(m), np.arange(m.size))
m = mass[np.logical_and(mask_isol, mask_shadow)]
ax[1].plot(np.sort(m), np.arange(m.size))


The above figure tells us that the unmatched subhalos have masses in the range $\lesssim 2 * 10^8 M_\odot$. With DM particle mass $\gtrsim 7 * 10^6 M_\odot$, this amounts to $\lesssim 29$ particles per subhalo. For such low particle numbers, the match-making is expected to fail.

Old observation, with n_link_ref == 15 and the number of particles in sub_srch selected strictly f_link_srch * n_srch, and not at least n_link_ref:

The above figure tells us that the unmatched subhalos have masses in the range $1\mathrm{-}3 * 10^8 M_\odot$. With DM particle mass $\gtrsim 7 * 10^7 M_\odot$, this amounts to $\lesssim 14\mathrm{-}43$ particles per subhalo. For such low particle numbers, the match-making is expected to fail, since 43/5<9 ~ 8, which is the minimum number of particles that need to be available for matching for a match to even be possible.